You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Preparation

In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
# if the dataset is not splitted it will be loaded as a single `train` set.
# `test` and `validation` sets are needed to be splitted manually.
dataset = load_dataset("csv", data_files="/content/bangla_sentiment_and_emotion_analysis.csv")

# remove columns that are not needed.
# keep these colmuns and remove the rest.
dataset = dataset.select_columns(["Review", "Emotion"])

# create a copy of the `Emotion` column for encoding it to integer representation
dataset["train"] = dataset["train"].add_column("Emotion_INT", dataset["train"]["Emotion"])

# column to stratify for splits
stratify_column = "Emotion_INT"

# convert/encode the column to ClassLabel object
dataset = dataset.class_encode_column(stratify_column)

#print(dataset['train'].features)
#print(dataset['train'].features['Emotion'].names)
#print(dataset['train'].features['Emotion'].str2int(['Happy', 'Anger']))

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Review', 'Emotion', 'Emotion_INT'],
        num_rows: 78130
    })
})


In [ ]:
# split the main dataset into `train` and `test` sets
train_with_test_valid = dataset["train"].train_test_split(test_size=0.2, stratify_by_column=stratify_column)

# split the `test` set from the newly splitted dataset into,
# 80% test and 20% evaluation sets
test_valid = train_with_test_valid["test"].train_test_split(test_size=0.2)

# combine everything into a single DatasetDict
dataset = DatasetDict({
    "train": train_with_test_valid["train"],
    "test": test_valid["train"],
    "eval": test_valid["test"]})

# remove the `Emotion_INT` column now that the dataset has been split
dataset = dataset.remove_columns("Emotion_INT")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Review', 'Emotion'],
        num_rows: 62504
    })
    test: Dataset({
        features: ['Review', 'Emotion'],
        num_rows: 12500
    })
    eval: Dataset({
        features: ['Review', 'Emotion'],
        num_rows: 3126
    })
})


In [ ]:
print("TRAIN:", dataset["train"][0])
print("TEST:", dataset["test"][0])
print("EVAL:", dataset["eval"][0])

print("\n", dataset["train"].features)

TRAIN: {'Review': 'Long lasting perfume. Performance is quite surprising. It smells too sweet and musky. It is not a regular used perfume. But it is price worthy for its performance. In winter it will perform well, I guess.', 'Emotion': 'Love'}
TEST: {'Review': 'thanks daraz 17 hours a product debar jonno', 'Emotion': 'Love'}
EVAL: {'Review': 'জিনিস অনুযায়ী দামটা একটু বেশি রাখা হয়েছে', 'Emotion': 'Sadness'}

 {'Review': Value(dtype='string', id=None), 'Emotion': Value(dtype='string', id=None)}


In [ ]:
# eos_token is used to mark the end-of-sentence or a piece of text in data
EOS_TOKEN = tokenizer.eos_token

# format prompt for `train` and `eval` sets
def format_train_prompt(data):
  text = data["Review"]
  emotion = data["Emotion"]

  prompts = []

  for text, emotion in zip(text, emotion):
    # must add EOS_TOKEN, otherwise your generation will go on forever!
    #text = f"""Classify the Text into Happy, Love, Sadness, Anger or Fear.
    text = f"""[INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: {text}
### Emotion: {emotion}""" + EOS_TOKEN

    prompts.append(text)

  return { "Prompt" : prompts, }

# format prompt for `test` set
def format_test_prompt(data):
  text = data["Review"]
  emotion = data["Emotion"]

  prompts = []

  for text, emotion in zip(text, emotion):
    # must add EOS_TOKEN, otherwise your generation will go on forever!
    text = f"""[INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: {text}
### Emotion:""" #+ EOS_TOKEN

    prompts.append(text)

  return { "Prompt" : prompts, }


# format the prompts for fine-tuning
dataset["train"] = dataset["train"].map(format_train_prompt, batched = True,)
dataset["eval"] = dataset["eval"].map(format_train_prompt, batched = True,)

dataset["test"] = dataset["test"].map(format_test_prompt, batched = True,)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Review', 'Emotion', 'Prompt'],
        num_rows: 62504
    })
    test: Dataset({
        features: ['Review', 'Emotion', 'Prompt'],
        num_rows: 12500
    })
    eval: Dataset({
        features: ['Review', 'Emotion', 'Prompt'],
        num_rows: 3126
    })
})


In [ ]:
print("TEST PROMPT:\n", dataset["test"][5]["Prompt"])
print("TEST EMOTION: ", dataset["test"][5]["Emotion"])

TEST PROMPT:
 [INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: Thanks ❤️
### Emotion:
TEST EMOTION:  Love


In [ ]:
!pip install evaluate

import evaluate
import numpy as np

In [ ]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels),
        "precision": precision_metric.compute(predictions=predictions, references=labels),
        }

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["eval"],
    dataset_text_field = "Prompt",
    compute_metrics=compute_metrics,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 120, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.5 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62,504 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.053100
2,2.337200
3,2.379600
4,1.835000
5,1.388300
6,1.127100
7,0.984500
8,0.757000
9,0.908500
10,0.749700


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

665.9666 seconds used for training.
11.1 minutes used for training.
Peak reserved memory = 6.273 GB.
Peak reserved memory for training = 1.773 GB.
Peak reserved memory % of max memory = 42.535 %.
Peak reserved memory for training % of max memory = 12.022 %.


In [ ]:
print(trainer_stats)

TrainOutput(global_step=120, training_loss=0.7190338095029195, metrics={'train_runtime': 665.9666, 'train_samples_per_second': 1.442, 'train_steps_per_second': 0.18, 'total_flos': 6462062218838016.0, 'train_loss': 0.7190338095029195, 'epoch': 0.015359017022910534})


In [ ]:
evaluation_stats = trainer.evaluate()

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.29 GiB. GPU 0 has a total capacity of 14.75 GiB of which 5.32 GiB is free. Process 159997 has 9.43 GiB memory in use. Of the allocated memory 9.16 GiB is allocated by PyTorch, and 121.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
evaluation_stats

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
print("TEST:",dataset["test"][7]["Prompt"])
print("TEST EMOTION:", dataset["test"][7]["Emotion"])

print()

print("TEST:",dataset["test"][10]["Prompt"])
print("TEST EMOTION:", dataset["test"][10]["Emotion"])

TEST: [INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: ঘড়িটা ভালো কিন্তু অর্ডার ককরেছিলাম স্পাইডার ম্যান দিসে বারবি ডল, তাই ফেরত দিয়েছি
### Emotion:
TEST EMOTION: Sadness

TEST: [INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: মশা একটু কম কামরায়, কাজ করে তো করেনা, প্রচুর পরিমাণে ক্লিম খরচ করতে হয়, এবং একটা চর্চাটে ভাব চলে আসে স্কিনে।।
### Emotion:
TEST EMOTION: Sadness


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
#inputs = tokenizer( [dataset["test"][14]["Prompt"] ], return_tensors = "pt").to("cuda")
inputs = tokenizer(
    [
        """[INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: jottoshob baje product
### Emotion:"""
    ],
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]Analyze the emotion of the text and determine if it is Happy, Love, Sadness, Anger or Fear, and return the answer as the corresponding emotion label 'Happy' or 'Love' or 'Sadness' or 'Anger' or 'Fear'.[/INST]
### Text: jottoshob baje product
### Emotion: Happy</s>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("emotion_analysis_mistral_7b") # Local saving
tokenizer.save_pretrained("emotion_analysis_mistral_7b")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

('emotion_analysis_mistral_7b/tokenizer_config.json',
 'emotion_analysis_mistral_7b/special_tokens_map.json',
 'emotion_analysis_mistral_7b/tokenizer.model',
 'emotion_analysis_mistral_7b/added_tokens.json',
 'emotion_analysis_mistral_7b/tokenizer.json')

In [ ]:
!tar -czvf emotion_analysis_mistral_7b_0.2.0_14_05_2024.tar.gz /content/emotion_analysis_mistral_7b

tar: Removing leading `/' from member names
/content/emotion_analysis_mistral_7b/
/content/emotion_analysis_mistral_7b/special_tokens_map.json
/content/emotion_analysis_mistral_7b/tokenizer.model
/content/emotion_analysis_mistral_7b/adapter_config.json
/content/emotion_analysis_mistral_7b/adapter_model.safetensors
/content/emotion_analysis_mistral_7b/README.md
/content/emotion_analysis_mistral_7b/tokenizer_config.json
/content/emotion_analysis_mistral_7b/tokenizer.json


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    ),
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q5_k_m", token = "")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 4.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.45 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:22<00:00,  2.59s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00003.bin...
Unsloth: Saving model/pytorch_model-00002-of-00003.bin...
Unsloth: Saving model/pytorch_model-00003-of-00003.bin...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be ./model-unsloth.F16.gguf
This will take 3 minutes...
INFO:convert:Loading model file model/pytorch_model-00001-of-00003.bin
INFO:convert:Loading model file model/pytorch_model-00001-of-00003.bin
INFO:convert:Loading model file model/pytorch_model-00002-of-00003.bin
INFO:convert:Loading model file model/pytorch_model-00003-of-00003.bin
INFO:convert:model parameters count : 7241732096 (7B)
INFO:convert:params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=32768, n_ff=14336, n_head=32, n_head_kv=8, n_exper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r '/content/model-unsloth.Q4_K_M.gguf' /content/drive/MyDrive/models/

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).